# Time Series Modeling
- First steps include:
    - Add date columns
    - impute missing electricity consumption values

In [27]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import timedelta
import pickle


In [28]:
pd.set_option('display.max_columns', None)

In [29]:
with open('berkeley_matched_readings.pkl','rb') as read_file:
    berkeley_df = pickle.load(read_file)

In [30]:
berkeley_df = berkeley_df.sort_values(by=['building_id', 'timestamp']).reset_index(drop=True)


### Add date, month, hour, day columns

In [31]:
berkeley_time = [str(rows) for index, rows in berkeley_df['timestamp'].iteritems()]    


In [32]:
# Function that converts timestamp to datetime, then creates hour, day, month, and date series

def to_datetime(time_list):

    new_time_list = []

    for i in time_list:
        i = i.strip()
        new_time_list.append(dt.datetime.strptime(i, '%Y-%m-%d %H:%M:%S'))
    
    hours = pd.Series(new_time_list).dt.hour
    days = pd.Series(new_time_list).dt.day
    months = pd.Series(new_time_list).dt.month
    years = pd.Series(new_time_list).dt.year
    date = pd.Series(new_time_list).dt.date
    
    return new_time_list, hours, days, months, years, date

In [33]:
berkeley_new_time, berkeley_hours, berkeley_days, berkeley_months, berkeley_years, berkeley_date = to_datetime(berkeley_time)


In [34]:
berkeley_df['timestamp'] = berkeley_new_time

In [35]:
berkeley_df.insert(2, column = 'Date', value = berkeley_date)
berkeley_df.insert(3, column = 'Month', value = berkeley_years)
berkeley_df.insert(4, column = 'Year', value = berkeley_months)
berkeley_df.insert(5, column = 'Day', value = berkeley_days)
berkeley_df.insert(6, column = 'Hour', value = berkeley_hours)

### Add season column

In [36]:
# Create and add season column:

def season_helper(month):
    if month in [12, 1, 2]:
        return 'Winter'
    if month in [3, 4, 5]:
        return 'Spring'
    if month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'


In [37]:
berkeley_df.insert(3, column = 'Season', value = berkeley_df['Month'].apply(season_helper))


In [38]:
berkeley_df

,building_id,timestamp,Date,Season,Month,Year,Day,Hour,meter_reading_scraped
0,565,2016-01-01 00:00:00,2016-01-01,Fall,2016,1,1,0,8.0
1,565,2016-01-01 01:00:00,2016-01-01,Fall,2016,1,1,1,7.0
2,565,2016-01-01 02:00:00,2016-01-01,Fall,2016,1,1,2,8.0
3,565,2016-01-01 03:00:00,2016-01-01,Fall,2016,1,1,3,8.0
4,565,2016-01-01 04:00:00,2016-01-01,Fall,2016,1,1,4,8.0
...,...,...,...,...,...,...,...,...,...
2539675,655,2019-11-30 19:00:00,2019-11-30,Fall,2019,11,30,19,127.0
2539676,655,2019-11-30 20:00:00,2019-11-30,Fall,2019,11,30,20,125.0
2539677,655,2019-11-30 21:00:00,2019-11-30,Fall,2019,11,30,21,121.0
2539678,655,2019-11-30 22:00:00,2019-11-30,Fall,2019,11,30,22,123.0


### Impute missing values
- This notebook only looks at buildings that having few missing values (less than 10 consecutive NaN values at a time)
- These values will be filled using ffill

In [49]:
# Split dataframe by the 74 buidlings.

def building_df_separator(full_df, building_no):
    
    building_df = full_df[full_df['building_id'] == building_no]
        
    return building_df

In [50]:
building_list = berkeley_df['building_id'].unique()

In [51]:
building_list

array([565, 567, 568, 569, 570, 571, 573, 574, 575, 576, 577, 580, 582,
       583, 584, 585, 586, 587, 588, 589, 592, 594, 595, 597, 598, 599,
       600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612,
       614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 626, 627,
       628, 630, 631, 632, 633, 634, 635, 636, 637, 638, 640, 641, 642,
       643, 644, 645, 646, 649, 652, 653, 654, 655])

In [54]:
# building_dict key: building number
# building_dict value: buidling's berkeley dataframe

building_dict = {}

for i in building_list:
    building_dict[i] = building_df_separator(berkeley_df, i)

In [55]:
building_dict[570]

,building_id,timestamp,Date,Season,Month,Year,Day,Hour,meter_reading_scraped
137280,570,2016-01-01 00:00:00,2016-01-01,Fall,2016,1,1,0,79.488
137281,570,2016-01-01 01:00:00,2016-01-01,Fall,2016,1,1,1,79.488
137282,570,2016-01-01 02:00:00,2016-01-01,Fall,2016,1,1,2,79.488
137283,570,2016-01-01 03:00:00,2016-01-01,Fall,2016,1,1,3,79.488
137284,570,2016-01-01 04:00:00,2016-01-01,Fall,2016,1,1,4,77.760
...,...,...,...,...,...,...,...,...,...
171595,570,2019-11-30 19:00:00,2019-11-30,Fall,2019,11,30,19,162.432
171596,570,2019-11-30 20:00:00,2019-11-30,Fall,2019,11,30,20,162.432
171597,570,2019-11-30 21:00:00,2019-11-30,Fall,2019,11,30,21,160.704
171598,570,2019-11-30 22:00:00,2019-11-30,Fall,2019,11,30,22,122.688


##### Find highest number of consecutive NaN values for each building

In [56]:
def na_count(na_dict):
    
    na_count_list = []
    
    for i in building_list:
        na_count = na_dict[i].meter_reading_scraped.isnull().astype(int).groupby(na_dict[i].meter_reading_scraped.notnull().astype(int).cumsum()).cumsum()
        na_count_list.append([i, na_count.sort_values(ascending=False).values[0]])
        
    return na_count_list
    

In [57]:
building_na_count = na_count(building_dict)

**Small NaN gap = less than 10 consecutive missing values**  
**Big NaN gap = 10 or more consecutive missing values**
- Reduce berkeley_df to only buildings with small NaN gap

In [58]:
big_na_gap = [i[0] for i in building_na_count if i[1] > 9]
small_na_gap = [i[0] for i in building_na_count if i[1] < 10]

In [59]:
big_or_small = []

for index, rows in berkeley_df['building_id'].iteritems():
    if rows in small_na_gap:
        big_or_small.append('Small')
    elif rows in big_na_gap:
        big_or_small.append('Big')

In [60]:
len(big_or_small)

2539680

In [61]:
berkeley_df['Big_or_Small'] = big_or_small

In [62]:
berkeley_small_df = berkeley_df[berkeley_df['Big_or_Small'] == 'Small']

In [63]:
building_small_dict = {}

for i in small_na_gap:
    building_small_dict[i] = building_df_separator(berkeley_small_df, i)

##### Replace NaN values in berkeley_small_df using ffill method

In [64]:
for keys, values in building_small_dict.items():
    values['meter_reading_scraped'] = values['meter_reading_scraped'].fillna(method='ffill')

/Users/andrewgraves/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [65]:
berkeley_small_clean = building_small_dict[565]

for i in small_na_gap[1:]:
    berkeley_small_clean = berkeley_small_clean.append(building_small_dict[i])
    

In [66]:
berkeley_small_df['meter_reading_scraped'].isna().value_counts()

False    1372778
True          22
Name: meter_reading_scraped, dtype: int64

In [67]:
berkeley_small_clean['meter_reading_scraped'].isna().value_counts()

False    1372800
Name: meter_reading_scraped, dtype: int64

In [80]:
small_na_index = berkeley_small_df[berkeley_small_df['meter_reading_scraped'].isna() == True].index


In [89]:
small_na_series = berkeley_small_clean['meter_reading_scraped'].loc[small_na_index]

In [90]:
small_na_series

34190         1.110974
155435      208.742400
509619      148.714280
509620      148.714280
509621      148.714280
509622      148.714280
509623      148.714280
510751       -9.925496
510752       -9.925496
510753       -9.925496
1082075      45.090908
1287995     544.933350
1322315      98.909090
1631195     258.400000
1768475     139.600000
1837115      76.400000
2317595    1612.091600
2351915      84.000000
2386235     385.636350
2420555     233.526370
2489195      56.666668
2523515     179.266660
Name: meter_reading_scraped, dtype: float64

In [91]:
# with open('small_na_series.pkl', 'wb') as to_write:
#     pickle.dump(small_na_series, to_write)